In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import glob
import os
import scipy.stats
from matplotlib import cm
import dask
import pickle

In [2]:
def get_pressure_weighted(x):
    dPref = (x.plev.values[0]-x.plev.values[-1])  #(p-ps)
    integral = []
    for i in range(len(x.plev)): #Integral of variable from P to Ps calculated as area between each pressure variable trapezoid then summed
        if i+1 < len(x.plev):
            area=((x.loc[dict(plev=x.plev.values[i])] + x.loc[dict(plev=x.plev.values[i+1])])/2)*(x.plev.values[i]-x.plev.values[i+1])
            integral.append(area)
    pw = (sum(integral))/dPref
    return(pw)

In [3]:
def low_pass_weights(window, cutoff):
    order = ((window - 1) // 2 ) + 1
    nwts = 2 * order + 1
    w = np.zeros([nwts])
    n = nwts // 2
    w[n] = 2 * cutoff
    k = np.arange(1., n)
    sigma = np.sin(np.pi * k / n) * n / (np.pi * k)
    firstfactor = np.sin(2. * np.pi * cutoff * k) / (np.pi * k)
    w[n-1:0:-1] = firstfactor * sigma
    w[n+1:-1] = firstfactor * sigma
    return w[1:-1]

In [4]:
wgts = low_pass_weights(41, 1/10)
weight = xr.DataArray(list(wgts), dims=['window'])

In [5]:
def jettracker(x):
    x = x.fillna(0)
    x = get_pressure_weighted(x)
    x = x.rolling(time=41, center=True).construct('window').dot(weight)
    x = x.dropna(dim='time',how='all')
    limit = np.quantile(x.values,0.9)
    x = x.where((x>=limit))
    x = x/x
    x = x.fillna(0)
    x = x.resample(time='QS-DEC').mean(dim='time',skipna=True)
    return x 

In [6]:
def get_files():
    models = glob.glob("/terra/data/cmip5/global/rcp85/*")
    avail={}
    for model in models:
        ua = glob.glob(str(model)+"/r1i1p1/day/2deg/ua_*")
        va = glob.glob(str(model)+"/r1i1p1/day/2deg/va_*")
        try:
            test = ua[0]
            avail[model.split('/')[-1]] = [ua,va]
        except:
             pass
    return avail

In [7]:
files = get_files()

In [ ]:
rcp85={}
for model in files:
    try:
        print(model)
        U = xr.open_mfdataset(files[model][0])
        V = xr.open_mfdataset(files[model][1])
        U = U.sel(plev= [85000,70000])
        U = U.sel(lat = slice(-75,0))
        U = U.sel(lon = slice(-50,30))
        U = U.sel(time=slice('2045', '2100'))
        V = V.sel(plev= [85000,70000])
        V = V.sel(lat = slice(-75,0))
        V = V.sel(lon = slice(-50,30))
        V = V.sel(time=slice('2045', '2100'))
        x = np.sqrt(np.square(U.ua.load()) + np.square(V.va.load()))
        rcp85[model] = jettracker(x)# , start = '2045', end ='2100')
    except:
        print(model,' - failed')

FGOALS-g2


In [ ]:
pickle.dump(rcp85, open( "rcp85_dic.p", "wb" ) )